HOMEWORK 2
ES 2.A

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

W = np.array([
    [0, 2/5, 1/5, 0, 0],
    [0, 0, 3/4, 1/4, 0],
    [1/2, 0, 0, 1/3, 0],
    [0, 0, 1/3, 0, 2/3],
    [0, 1/3, 0, 1/3, 0]
])

w = np.sum(W, axis=1)
D = np.diag(w)
# contruct the P matrix,  P = D^-1 * Lambda
P = np.linalg.inv(D) @ W

# global clock
w_star = np.max(w)
# compute the off-diagonal part of P_bar
P_bar = W/w_star
# add the diagonal part
P_bar = P_bar + np.diag(np.ones(len(w))-np.sum(P_bar,axis=1))

n_states = W.shape[0]

PARTICLES PERSPECTIVE

In [ ]:
import numpy as np

# define simulation parameters
n_particles = 1
n_simulations = 100
n_steps = 1000

# return node is b
return_node = 2

return_times_list = []
for _ in range(n_simulations):

    # pos tracks position of each particle in time
    pos = np.zeros((n_particles,n_steps), dtype=int)-1
    # every particle starts in b
    pos[: , 0] = return_node

    # save for each particle the transition time
    transition_times = np.zeros((n_particles,n_steps))

    # at each step move a particle
    for t in range(1, n_steps):
        for particle in range(n_particles):
            # don't execute this simulation if this particle has already returned
            if (pos[particle , t-1] == return_node and t-1 != 0) or pos[particle , t-1] == -1:
                continue

            # save position
            pos[particle, t] = np.random.choice(n_states, p=P[pos[particle, t-1],:])

            # compute the waiting time to the next transition
            # t_next = -log(u)/r
            # NOTE: we use the rate w[pos[t]] of the clock of the current position of each particle.
            # So it's a Local Clock for each particle
            t_next = -np.log(np.random.rand()) / w[pos[particle, t-1]]

            # save time
            transition_times[particle, t] = transition_times[particle, t-1] + t_next


    return_times = []
    for particle in range(n_particles):
        # find index of pos where particle returned to b
        index_return_node = np.where(pos[particle, :] == return_node)[0]
        # if particle never returned skip this result
        if len(index_return_node) == 2:
            # add transitoin time to return time list
            return_times.append(transition_times[particle , index_return_node[1] ] )

    # save only the time of the first particle that return to b, so the min between all times
    return_times_list.append(np.min(return_times))

mean_return_time_overall = np.mean(return_times_list)

print(f"Average time on {n_simulations} simulations: {mean_return_time_overall}")

HOMEWORK 2
ES 2.B

NODES PERSPECTIVE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# define simulaation parameters
n_particles = 100
n_simulations = 10
n_steps = 6500
end_time = 60

# start from node o
start_node = 0

return_times_list = []
tot_particles_in_node = np.zeros((n_states,n_simulations))
for simulation in range(n_simulations):
    # save how many particles are in node i at each time
    particles_in_node = np.zeros((n_states,n_steps), dtype=int)
    # initialize all particles in node o
    particles_in_node[start_node , 0] = n_particles

    # save when the transition happen
    transition_times = np.zeros(n_steps)

    node = start_node
    for t in range(1, n_steps):
        # chose a randomly from which node move a particle,
        # proportionally to the number of particles in each node
        # probability of chose node i = particles in node i / tot particles
        node = np.random.choice(n_states, p=particles_in_node[:, t-1]/n_particles)

        # chose where the particle should move
        # NOTE it could stay in the same node (we use P_bar instead of P)
        next_node = np.random.choice(n_states, p=P_bar[node,:])

        # update (if necessary) how many particles are in a node
        particles_in_node[:, t] = particles_in_node[:, t-1]
        if node != next_node:
            particles_in_node[node, t] -= 1
            particles_in_node[next_node, t] += 1

        # compute the waiting time to the next transition
        # t_next = -log(u)/r
        # NOTE: we use the rate 100 as suggested
        # So it's a Global Clock
        rate = n_particles
        t_next = -np.log(np.random.rand()) / rate
        transition_times[t] = transition_times[t-1] + t_next

        # stop after 60 time units
        if transition_times[t] > end_time:
            break

    tot_particles_in_node[:, simulation] = particles_in_node[:,t]

# compute mean particles in each node
mean_particles_in_node = np.zeros(n_states)
mean_particles_in_node = [ np.mean(tot_particles_in_node[node,:]) for node in range(n_states) ]

print(f"Average time on {n_simulations} simulations: {mean_particles_in_node}")

In [ ]:
# plot mean particle for each node during time
for node in range(n_states):
    plt.plot(particles_in_node[node,:])
    plt.title(f'node {node}')
    plt.show()